In [2]:
import numpy as np
import cv2

In [3]:
imgIn = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
], np.uint8)

In [4]:
histY = cv2.reduce(imgIn, 1, cv2.REDUCE_AVG).reshape(-1)
histY

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [41]:
histY = np.sum(imgIn, axis=1)
histY

array([ 0,  0,  0,  0,  1,  3,  5,  7,  9, 11, 10,  8, 10,  9,  7,  5,  3,
        1,  0,  0,  0,  0], dtype=uint64)

In [37]:
mask = imgIn != 0
ln = imgIn.shape[1]
ln - np.flip(mask, axis = 1).argmax(axis=1) - 1

array([10, 10, 10, 10,  5,  6,  7,  8,  9, 10, 10, 10, 10,  9,  8,  7,  6,
        5, 10, 10, 10, 10])

In [40]:
mask = histY != 0
ln = histY.shape[0]
ln - np.flip(mask, axis=0).argmax(axis=0) - 1

17

In [42]:
def first_nonzero(arr, axis=None):
    return(arr!=0).argmax(axis=axis or 0)


def last_nonzero(arr, axis=None):
    ax = axis or 0
    mask = arr!=0
    ln = arr.shape[ax] 
    val = ln - np.flip(mask, axis=ax or 0).argmax(axis=ax) - 1
    return val if axis is None else np.where(mask.any(axis=ax), val, -1)

In [43]:
first_nonzero(histY)

4

In [44]:
last_nonzero(histY)

17

In [45]:
def getLengths(C, imgIn):
    (h, w) = imgIn.shape[0:2]
    left = first_nonzero(imgIn, axis=1)
    right = last_nonzero(imgIn, axis=1)
    return np.transpose(right - left + 1)

In [46]:
lengths = getLengths(None, imgIn)
lengths

array([ 0,  0,  0,  0,  1,  3,  5,  7,  9, 11, 11, 11, 11,  9,  7,  5,  3,
        1,  0,  0,  0,  0])

In [110]:
def correctWithLengths(C, lengths, hist):
    histC = hist.copy().astype(float)
    histC[lengths > 0] = hist[lengths > 0] / lengths[lengths > 0]
    return np.rint(histC).astype(np.uint8)

In [111]:
histC = correctWithLengths(None, lengths, histY)
histC

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
      dtype=uint8)

# Rolling mean

In [114]:
signal = np.array([
    1, 1, 1, 1, 1, 1, 1, 1,
    100, 100, 
    1, 1, 1, 1, 1, 1, 1, 1,
    100, 100, 
    1, 1, 1, 1, 1, 1, 1, 1,
    100, 100, 100, 100, 100, 100, 100, 100, 
    1, 1, 1, 1, 1, 1, 1, 1,
    100, 100, 
    1, 1, 1, 1, 1, 1, 1, 1,
    100, 100, 
])
window = np.ones(3)

In [115]:
np.convolve(signal, window, 'valid')

array([  3.,   3.,   3.,   3.,   3.,   3., 102., 201., 201., 102.,   3.,
         3.,   3.,   3.,   3.,   3., 102., 201., 201., 102.,   3.,   3.,
         3.,   3.,   3.,   3., 102., 201., 300., 300., 300., 300., 300.,
       300., 201., 102.,   3.,   3.,   3.,   3.,   3.,   3., 102., 201.,
       201., 102.,   3.,   3.,   3.,   3.,   3.,   3., 102., 201.])

In [116]:
from scipy.signal import medfilt

In [118]:
medfilt(signal, 5)

array([  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1., 100., 100., 100., 100., 100.,
       100., 100., 100.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.])